# 二創 Scenario

## 1. Initialize agent

In [24]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# WARNING: You should use environment variables for this
# TODO: Make env variables accessible through juypter notebooks
API_KEY = "alice_api_123456789"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL, api_key=API_KEY)

agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST, 
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

IPython autoawait is `on`, and set to use `asyncio`


## 2. DID exchange 

### **Build listener for DID exchange**

In [2]:
my_did = await agent_controller.wallet.get_public_did()
my_did = my_did['result']['did']
print("my DID", my_did)
sp_did = 'None'
music_did = 'None'

def messages_handler(payload):
    global sp_did
    global music_did
    connection_id = payload["connection_id"]
    did = payload["content"]
    if did[:3] == 'sp_':
        sp_did = did[3:]
        print('sp_did', sp_did)
    else:
        music_did = did[6:]
        print('music_did', music_did)
        
    asyncio.get_event_loop().create_task(agent_controller.messaging.send_message(connection_id, my_did))
    print("Handle message", payload, connection_id)
    
def connection_handler(payload):
    print("Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    
    
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}
message_listener = {
    "handler": messages_handler,
    "topic": "basicmessages"
}

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

agent_controller.register_listeners([message_listener, connection_listener], defaults=False)

my DID PQRXDxdGqQGSZ8z69p4xZP
Subscribing too: basicmessages
Subscribing too: connections


### Build DM-SP connection

In [19]:
# Create Invitation
invite = await agent_controller.connections.create_invitation()
connection_id = invite["connection_id"]
invite_message = invite['invitation']
print("Connection ID", connection_id)
print("#######################")
print("Invitation - Copy this!!")
print(invite_message)

Connection ID 2667ae6d-128e-4d6d-ac81-e93c3df04bec
#######################
Invitation - Copy this!!
{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': 'dcce54a9-996e-480e-a3a6-cd1b8b82e8ca', 'label': 'Alice', 'recipientKeys': ['Av6mvcqbEhVCMLLNNEq3oQimyT2nBMixSKqT1K8LshxB'], 'serviceEndpoint': 'http://192.168.65.3:8020'}
Connection Handler Called
Connection 2667ae6d-128e-4d6d-ac81-e93c3df04bec in State invitation
Connection Handler Called
Connection 2667ae6d-128e-4d6d-ac81-e93c3df04bec in State request


### > Break here - switch to SP to receive invitation <

In [20]:
# Accept Request for Invite created
connection = await agent_controller.connections.accept_request(connection_id)
print("ACCEPT REQUEST")
print(connection)
print("state", connection["state"])

Connection Handler Called
Connection 2667ae6d-128e-4d6d-ac81-e93c3df04bec in State response
ACCEPT REQUEST
{'state': 'response', 'rfc23_state': 'response-sent', 'created_at': '2023-01-05 12:02:55.796619Z', 'connection_id': '2667ae6d-128e-4d6d-ac81-e93c3df04bec', 'their_did': 'GvyYisyaWnttacfyKYnkYq', 'their_role': 'invitee', 'my_did': 'UM3XjxkgJzJneYsgbJrHtc', 'invitation_key': 'Av6mvcqbEhVCMLLNNEq3oQimyT2nBMixSKqT1K8LshxB', 'invitation_mode': 'once', 'updated_at': '2023-01-05 12:03:20.212212Z', 'routing_state': 'none', 'accept': 'manual', 'their_label': 'Bob'}
state response


In [21]:
trust_ping = await agent_controller.messaging.trust_ping(connection_id, "hello")
print("Trust Ping", trust_ping)

Trust Ping {'thread_id': '550b2695-4c49-44f2-845e-aceb93db8729'}
Connection Handler Called
Connection 2667ae6d-128e-4d6d-ac81-e93c3df04bec in State active


In [22]:
#####################
# get SP connection #
#####################

connection = await agent_controller.connections.get_connection(connection_id)
print(connection)
print("Is Active?", connection["state"])
sp_connection = connection["connection_id"]
print(sp_connection)

{'state': 'active', 'rfc23_state': 'completed', 'created_at': '2023-01-05 12:02:55.796619Z', 'connection_id': '2667ae6d-128e-4d6d-ac81-e93c3df04bec', 'their_did': 'GvyYisyaWnttacfyKYnkYq', 'their_role': 'invitee', 'my_did': 'UM3XjxkgJzJneYsgbJrHtc', 'invitation_key': 'Av6mvcqbEhVCMLLNNEq3oQimyT2nBMixSKqT1K8LshxB', 'invitation_mode': 'once', 'updated_at': '2023-01-05 12:03:21.880973Z', 'routing_state': 'none', 'accept': 'manual', 'their_label': 'Bob'}
Is Active? active
2667ae6d-128e-4d6d-ac81-e93c3df04bec
music_did gZj39Y8JGQ5GVQp2y6zFH
Handle message {'connection_id': '2667ae6d-128e-4d6d-ac81-e93c3df04bec', 'message_id': '29a2097e-7f9d-406b-9e92-7e0943b129b8', 'content': 'music_gZj39Y8JGQ5GVQp2y6zFH', 'state': 'received'} 2667ae6d-128e-4d6d-ac81-e93c3df04bec
music_did gZj39Y8JGQ5GVQp2y6zFH
Handle message {'connection_id': '2667ae6d-128e-4d6d-ac81-e93c3df04bec', 'message_id': '73ea0cef-1389-41da-adbb-65a1250f5027', 'content': 'music_gZj39Y8JGQ5GVQp2y6zFH', 'state': 'received'} 2667ae6d-

### > Break here and go to SP to send message, after finishing that, come back <

------

### Build DM-MUSIC connection

In [34]:
# Create Invitation
invite = await agent_controller.connections.create_invitation()
connection_id = invite["connection_id"]
invite_message = invite['invitation']
print("Connection ID", connection_id)
print("#######################")
print("Invitation - Copy this!!")
print(invite_message)

Connection ID 54801036-1d74-4346-9c73-a14d43d36c86
#######################
Invitation - Copy this!!
{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '3082ab84-7846-435c-b036-35065edd3249', 'label': 'Alice', 'recipientKeys': ['Ea5qcPR9wGHhzCVAPUp3qVHr3QtNWS7WiWvGmoRcud2t'], 'serviceEndpoint': 'http://192.168.65.3:8020'}


### > Break here - switch to MUSIC to receive invitation <

In [35]:
# Accept Request for Invite created
connection = await agent_controller.connections.accept_request(connection_id)
print("ACCEPT REQUEST")
print(connection)
print("state", connection["state"])

ACCEPT REQUEST
{'state': 'response', 'rfc23_state': 'response-sent', 'created_at': '2023-01-05 12:05:33.799199Z', 'connection_id': '54801036-1d74-4346-9c73-a14d43d36c86', 'their_did': 'Dw4EQ7TFHaKC1J5rc2tSgc', 'their_role': 'invitee', 'my_did': '4doDed3C8ohLwYi6L4g2fN', 'invitation_key': 'Ea5qcPR9wGHhzCVAPUp3qVHr3QtNWS7WiWvGmoRcud2t', 'invitation_mode': 'once', 'updated_at': '2023-01-05 12:05:56.267239Z', 'routing_state': 'none', 'accept': 'manual', 'their_label': 'Bob'}
state response


In [38]:
trust_ping = await agent_controller.messaging.trust_ping(connection_id, "hello")
print("Trust Ping", trust_ping)

Trust Ping {'thread_id': '64bd9c15-bfaf-4f4a-a26a-053663cc7c9f'}


In [39]:
#####################
# get MUSIC connection #
#####################

connection = await agent_controller.connections.get_connection(connection_id)
print(connection)
print("Is Active?", connection["state"])
music_connection = connection["connection_id"]
print(music_connection)

{'state': 'active', 'rfc23_state': 'completed', 'created_at': '2023-01-05 12:05:33.799199Z', 'connection_id': '54801036-1d74-4346-9c73-a14d43d36c86', 'their_did': 'Dw4EQ7TFHaKC1J5rc2tSgc', 'their_role': 'invitee', 'my_did': '4doDed3C8ohLwYi6L4g2fN', 'invitation_key': 'Ea5qcPR9wGHhzCVAPUp3qVHr3QtNWS7WiWvGmoRcud2t', 'invitation_mode': 'once', 'updated_at': '2023-01-05 12:06:06.559023Z', 'routing_state': 'none', 'accept': 'manual', 'their_label': 'Bob'}
Is Active? active
54801036-1d74-4346-9c73-a14d43d36c86


### > Break here and go to MUSIC to send message, after finishing that, come back <

In [13]:
print(music_did)
print(sp_did)

None
None
sp_did X8HWQruWXrv8s6BBLXm53B
Handle message {'connection_id': 'ce707209-32a6-4099-b26b-924e337e6d68', 'message_id': '09a50eeb-6cee-4e7b-8d9b-213a25f788a7', 'content': 'sp_X8HWQruWXrv8s6BBLXm53B', 'state': 'received'} ce707209-32a6-4099-b26b-924e337e6d68
music_did gZj39Y8JGQ5GVQp2y6zFH
Handle message {'connection_id': '6be95fa2-173c-498b-9e5d-a1107802bb1d', 'message_id': 'dfdb0da0-f6a3-47b4-8de6-2eb932070b1b', 'content': 'music_gZj39Y8JGQ5GVQp2y6zFH', 'state': 'received'} 6be95fa2-173c-498b-9e5d-a1107802bb1d
music_did gZj39Y8JGQ5GVQp2y6zFH
Handle message {'connection_id': '6be95fa2-173c-498b-9e5d-a1107802bb1d', 'message_id': '1859d348-0ce4-4bfa-ac74-e54cbb9edd2b', 'content': 'music_gZj39Y8JGQ5GVQp2y6zFH', 'state': 'received'} 6be95fa2-173c-498b-9e5d-a1107802bb1d


## 4. Give SP the VC of re-create right - DM
Scenario:     
When SP pay for the re-create right, the SP can show DM the certificate.     
On receiving the certificate, the DM will give SP the VC, so that the Music platform will recognize the VC and allow it to put VC on the platform


### SP (re-creater) re-creates the song, this is the info of the song it recreates on (These information is fetched from 3. in SP)

In [ ]:
target_song = "F4zojhjnUqDAAzYsdpqmtQ" #### !! Copy the music ID from sp_scenario3.ipynb - Verify the result back !! ##
origin = "PQRXDxdGqQGSZ8z69p4xZP"

### Write Ownership

In [ ]:
# Define you schema name - must be unique on the ledger
schema_name = "recreate_schema"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema
attributes = ["origin", "target", "time", "owner"]

# Write schema to ledger
response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
schema_id = response["schema_id"]
print(schema_id)

# Write cred. def.
response = await agent_controller.definitions.write_cred_def(schema_id)
cred_def_id = response["credential_definition_id"]
print(cred_def_id)




### Write VC

In [ ]:
# Get nym 
nym = await agent_controller.wallet.get_public_did()
print(nym["result"]["did"])

# get schema id
created_schema = await agent_controller.schema.get_created_schema(schema_issuer_did='PQRXDxdGqQGSZ8z69p4xZP')
print(created_schema)
schema_id = created_schema["schema_ids"][0]

In [ ]:
# get cred_def_id
definitions = await agent_controller.definitions.search_created(schema_id=schema_id)
print(definitions["credential_definition_ids"])
cred_def_id = definitions["credential_definition_ids"][0]

In [ ]:
response = await agent_controller.connections.get_connections()
results = response['results']
print(len(results))
# print("Results : ", results)
if len(results) > 0:
    connection = response['results']
    for con in connection:
        print("Connection :", con)
        if con['state'] == 'active' and con["their_label"] == "Carol":        
            connection_id = con["connection_id"]
            print("Active Connection ID : ", connection_id)
        else: print("Inactive Connection ID: ", con["connection_id"], "with state: ", con["state"])
else:
    print("You must create a connection")

In [ ]:
import datetime
time = datetime.datetime.now().strftime("%m-%d-%Y, %H:%M:%S")
credential_attributes = [
    {"name": "target", "value": target_song},
    {"name": "time", "value": time},
    {"name": "origin", "value": origin},
    {"name": "owner", "value": sp_did},
]
print(credential_attributes)


# send credential
record = await agent_controller.issuer.send_credential(connection_id, schema_id, cred_def_id, credential_attributes, auto_remove=False, trace=True)
record_id = record['credential_exchange_id']
state = record['state']
role = record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")

### > Break: Go to SP for credential exchange <

### Get records

In [ ]:
response = await agent_controller.issuer.get_records()
print(response['results'])
cred_record = await agent_controller.issuer.get_record_by_id(record_id)
state = cred_record['state']
role = cred_record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")

In [ ]:
## Remove records
response = await agent_controller.issuer.remove_record(record_id)
print(response)

# Terminate Agent

In [23]:
response = await agent_controller.terminate()
print(response)

None
